# Module 1 Tutorial

There are numerous open-source libraries, collections of functions, that have been developed in Python that we will make use of in this course.

The first one is called NumPy and you can find the documentation [here](https://numpy.org/). It is one of the most widely-used libraries for scientific computating in python. The second library we will use will be a module from Scipy, called scipy.stats ([scipy.stats documentation](https://docs.scipy.org/doc/scipy/reference/stats.html)), and the third is a library for handling database-like structures called Pandas for which you can find the documentation at this link: [Pandas documentation](https://pandas.pydata.org/docs/user_guide/index.html).

We import the libraries with the following statement:

In [1]:
import numpy
from scipy import stats
import pandas

Now we will start building our toolbox with some simple tools to describe our data: 

## Confidence Intervals and Descriptive Statistics

In module 1 of the course the first thing that is covered is confidence intervals. As we only have access to samples of data we assume that neither the population mean or the population standard deviation are known and we work with point estimates, sample mean, and sample standard deviation (also called standard error).

To build a confidence interval we must specify a confidence level and provide the sample of our data. 

Below is a simple function to obtain the confidence interval of your sample.

In [2]:
def get_confidence_interval(data, confidence=0.95):
    """ Determines the confidence interval for a given set of data, 
        assuming the population standard deviation is not known.

    Args:  # 'arguments', or inputs to the function
        data (single-column or list): The data
        confidence (float): The confidence level on which to produce the interval.

    Returns:
        c_interval (tuple): The confidence interval on the given data (lower, upper).
    """

    n = len(data)  # determines the sample size
    m = numpy.mean(data)  # obtains mean of the sample

    se = stats.sem(data)  # obtains standard error of the sample

    c_interval = stats.t.interval(confidence, n-1, m, se)  # determines the confidence interval
    return c_interval  # which is of the form (lower bound, upper bound)

We can walk through the function above:
The name of the function is *get_confidence_interval* and the function takes two arguments, the first is the sample that you are interested in calculating the confidence interval for, and the second is the desired confidence level. The second argument is optional and will default to 95% if not specified. 95% is a very typical confidence level used in most applications.

Inside the function we first obtain *n*, the sample size. Then we calculate the sample mean using the numpy.mean function ([numpy.mean documentation](https://numpy.org/doc/stable/reference/generated/numpy.mean.html)), and the sample standard error with the scipy.stats.sem function ([scipy.stats.mean documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html)). 
Finally, we calculate the confidence interval using the scipy.stats.t.interval function ([scipy.stats.t documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.t.html)), this function needs the desired confidence level, the degrees of freedom, the sample mean, and the standard error, in order to calculate the upper and lower bounds of the confidence interval.

Let us illustrate this function with Example 12.6 from the course book: in this example both $\mu$ and $\sigma$, the population parameters are unknown. The sample data is given as {1, 0, 5} and the question asks for the 99% confidence interval for $\mu$ assuming a normally distributed population.

This is easily calculated using the function we defined above:

In [3]:
sample_data = [1, 0, 5]
get_confidence_interval(sample_data, confidence=0.99)

(-13.160448409591625, 17.160448409591623)

Another tool that could be useful in order to help us understand our data is provided by the pandas library. The .describe() function produces a statistical description of our sample. In order to call this function however our data needs to be in a pandas.Series or pandas.DataFrame object, and we need a column for each group we want to describe.

Let's say we have some data stored in two columns labeled "Day" and "Temperature", which contains the temperature readings at 6 set times over the course of two different days:

In [4]:
sample_dataframe = pandas.DataFrame(
    {
        "Day": ["Day 1"]*6 + ["Day 2"]*6,
        "Temperature": [15, 17, 19, 19, 18, 16, 14, 15, 18, 19, 21, 18]
    }
)

The DataFrame looks like this:

In [5]:
sample_dataframe

,Day,Temperature
0,Day 1,15
1,Day 1,17
2,Day 1,19
3,Day 1,19
4,Day 1,18
5,Day 1,16
6,Day 2,14
7,Day 2,15
8,Day 2,18
9,Day 2,19


Now, because we created the DataFrame we know what is in it. But if we didn't, we could ask what the columns are:

In [6]:
column_names = sample_dataframe.columns
column_names

Index(['Day', 'Temperature'], dtype='object')

We can assign these column names as either independent or dependent to help us keep track of what we're doing:

In [7]:
independent_col = column_names[0]
dependent_col = column_names[1]

And we can also see how many values our independent variable takes and what they are by using the pandas.unique function:

In [8]:
independent_variable_values = pandas.unique(sample_dataframe[independent_col])
independent_variable_values

array(['Day 1', 'Day 2'], dtype=object)

Now, we want to separate the samples corresponding to each independent variable and obtain a statistical description of them.

We do this as follows:

In [9]:
dependent_variable_data = pandas.DataFrame(columns=[day for day in independent_variable_values])

This is equivalent to pandas.DataFrame(columns=["Day 1", "Day 2"]), but the code in the block above would automatically create an additional column for any more days added to the dataset.
Right now the DataFrame looks like this:

In [10]:
dependent_variable_data

,Day 1,Day 2


Let's put the correct data into it now.

It looks complicated, but don't worry too much. 
If we unpack the lines below:
1. sample_dataframe[dependent_col] selects the data in the dependent variable column.
2. [sample_dataframe[independent_col]==independent_variable_values[0]] selects all the data where independent_col (Day) is equal to a specific value, independent_variable_values[0], ('Day 1').
3. The final .reset_index(drop) ensures that the selected data does not retain a label showing its index in the original file.

In [11]:
dependent_variable_data["Day 1"] = sample_dataframe[dependent_col][sample_dataframe[independent_col]==independent_variable_values[0]].reset_index(drop=True)
dependent_variable_data["Day 2"] = sample_dataframe[dependent_col][sample_dataframe[independent_col]==independent_variable_values[1]].reset_index(drop=True)

Just to be clear, the following is equivalent, but less general:

In [12]:
dependent_variable_data["Day 1"] = sample_dataframe["Temperature"][sample_dataframe["Day"]=="Day 1"].reset_index(drop=True)
dependent_variable_data["Day 2"] = sample_dataframe["Temperature"][sample_dataframe["Day"]=="Day 2"].reset_index(drop=True)

The data now looks like this:

In [13]:
dependent_variable_data

,Day 1,Day 2
0,15,14
1,17,15
2,19,18
3,19,19
4,18,21
5,16,18


We can now request a statistical description of each column from our dataset:

In [14]:
sample_statistics = dependent_variable_data.describe()
sample_statistics

,Day 1,Day 2
count,6.000000,6.000000
mean,17.333333,17.500000
std,1.632993,2.588436
min,15.000000,14.000000
25%,16.250000,15.750000
50%,17.500000,18.000000
75%,18.750000,18.750000
max,19.000000,21.000000


And what we see returned is the sample size, the mean of our sample, the standard deviation (which is not of great use, can you explain why?), the minumum, maximum, and different percentiles. We can access the different information from each column by name or by index:

In [15]:
print(sample_statistics["Day 1"]["mean"])

print(sample_statistics["Day 1"][1])

17.333333333333332
17.333333333333332


We can now move on to the next part of module 1.

## Hypothesis Testing
We have already constructed a set of sample data to test our functions with, we have one indepedendent variable (Day) which takes on two different values (Day 1 and Day 2), and we have the temperature on those days as our dependent variables.
The question we could ask now is, is the mean temperature of the two days statistically different? We can write this as a hypothesis test:

$H_0 : \mu_1 - \mu_2 = 0$\
$H_1 : \mu_1 - \mu_2 \neq 0$

The independent sample t-test can be used to test this hypothesis. But, an underlying assumption of the independent samples t-test is that the two populations being compared have equal variances. 

The test for equal variance can be written as another hypothesis test and is commonly called the Levene test:

$H_0 : \sigma^2_1 - \sigma^2_2 = 0$\
$H_1 : \sigma^2_1 - \sigma^2_2 \neq 0$

So let's add to our toolbox again. 
This time we are not writing our own function immediately, but first using a function from the stats library. This library includes the function stats.levene which performs the levene test. 

The Levene test function takes the data from each group and returns an $F$ and $p$ value. Depending on the desired significance level, we can then accept or reject the Levene test.

In [16]:
stats.levene(dependent_variable_data["Day 1"], dependent_variable_data["Day 2"])

LeveneResult(statistic=0.4245283018867924, pvalue=0.5293748457291648)

If our significance level, $\alpha$, is 0.05 (a common value), we can observe that in this case the $p$ value is larger than $\alpha$ and so we fail to reject the null hypothesis: we cannot statistically discount the possibility that the two samples have equal variance at this level of significance.

We now want to adress the question of equal means. Let's add the t-test to our toolbox:

In [17]:
def t_test(data_group1, data_group2, confidence=0.95):
    alpha = 1-confidence

    if stats.levene(data_group1, data_group2)[1]>alpha:
        equal_variance = True
    else:
        equal_variance = False

    t, p = stats.ttest_ind(data_group1, data_group2, equal_var = equal_variance)

    reject_H0 = "True"
    if p > alpha:
        reject_H0 = "False"

    return({'t': t, "p": p, "Reject H0": reject_H0})

Our function to perform the $t$-test is called "t-test" and it takes three possible inputs:
1. The data of the first column (or group) that would correspond to $\mu_1$ in the hypothesis test, 
2. The data of the second column (or group) that would correspond to $\mu_2$ in the hypothesis test, and finally, 
3. The desired confidence level, this will default to 95% if not specified.

Inside the function, the confidence level is used to determine the $\alpha$ value which is the significance level for the $t$-test.\
Then, the Levene test (which we discussed previously) is run to determine if the two groups have equal variance or not. This is done because the function that performs the $t$-test, [stats.ttest_ind](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html), needs this as an input; it modifies the calculations based on whether or not the two groups have equal variance.

So, after the Levene test we calculate the $t$ value and $p$ value of the $t$-test. The inputs to "stats.ttest_ind" are the data for the first group, the data for the second group, and the results of the Levene test.

Finally, we check if $p$ is larger than our desired significance level.

Let us illustrate this for our previous temperature dataset:

In [18]:
t_test(dependent_variable_data["Day 1"], dependent_variable_data["Day 2"], confidence=0.95)

{'t': -0.133392632128053, 'p': 0.8965290851338013, 'Reject H0': 'False'}

The outputs from our function are the $t$ value, the $p$ value, and whether or not we accept the null hypothesis.
We see that because the $p$ value is larger than our confidence level, we fail to reject the null hypothesis.